# Introduction to the Keras Tuner

This tutorial follows through the hyperparameter tuning (hypertuning), working on the Fashion MNIST dataset. There are two kinds of hyperparameters:
1. **Model hyperparameters**, influencing model selection (_e.g._ number / width of hidden layers)
2. **Algorithm hyperparameters**, influencing speed, quality, _etc_. of the algorithm (_e.g._ the learning rate or the number of nearest neighbours for a k Nearest Neighbour classifier).

Let's start by importing the necessary libraries and load and prepare the data.

In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

In [2]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [3]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

### Define the model
We define the model by defining a model builder function to define the image classification model.  

In [4]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model

## Procedure
First, we instantiate the tuner and perform hypertuning. There are four tuners available: `RandomSearch`, `Hyperband`, `BayesianOptimization` and `Sklearn`. We use `Hyperband`. We need also to define what objective to optimize (_e.g._ in this example `'val_accuracy'`) and the maximal number of epochs to train.

In [6]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3)

We create a callback to stop early after reaching a certain value for the validation loss.

In [7]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

And now it's time to run the hyperparameters search. The parameters are the same as when working with `tf.keras.model.fit`:

In [8]:
tuner.search(img_train, 
             label_train, 
             epochs=50, 
             validation_split=0.2, 
             callbacks=[stop_early],
             directory='keras-tuning')


Trial 30 Complete [00h 00m 58s]
val_accuracy: 0.8811666369438171

Best val_accuracy So Far: 0.8870000243186951
Total elapsed time: 00h 10m 26s
INFO:tensorflow:Oracle triggered exit


AttributeError: 'Hyperband' object has no attribute 'get_best_hyperparametrs'

In [9]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected layer
is {best_hps.get('units')} and the optimal learning_rate for the optimizer is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected layer
is 256 and the optimal learning_rate for the optimizer is 0.001.



### Train the model

In [10]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

Epoch 1/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5035 - accuracy: 0.8220 - val_loss: 0.4186 - val_accuracy: 0.8480
Epoch 2/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3779 - accuracy: 0.8622 - val_loss: 0.3758 - val_accuracy: 0.8607
Epoch 3/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3385 - accuracy: 0.8761 - val_loss: 0.3360 - val_accuracy: 0.8767
Epoch 4/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3110 - accuracy: 0.8857 - val_loss: 0.3356 - val_accuracy: 0.8775
Epoch 5/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2905 - accuracy: 0.8911 - val_loss: 0.3380 - val_accuracy: 0.8793
Epoch 6/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2743 - accuracy: 0.8981 - val_loss: 0.3260 - val_accuracy: 0.8812
Epoch 7/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2633 - accuracy: 0.9011 - val_loss: 0.3293 - val_accuracy:

In [11]:
val_accuracy = history.history['val_accuracy']
best_epoch = val_accuracy.index(max(val_accuracy)) + 1
print('Best epoch: %d' % (best_epoch,))

Best epoch: 34


And now we retrain the model with the best number of epochs according to the test we just performed!

In [12]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/34
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4973 - accuracy: 0.8235 - val_loss: 0.4229 - val_accuracy: 0.8446
Epoch 2/34
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3746 - accuracy: 0.8641 - val_loss: 0.3707 - val_accuracy: 0.8693
Epoch 3/34
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3327 - accuracy: 0.8779 - val_loss: 0.3684 - val_accuracy: 0.8673
Epoch 4/34
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3098 - accuracy: 0.8862 - val_loss: 0.3358 - val_accuracy: 0.8810
Epoch 5/34
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2891 - accuracy: 0.8941 - val_loss: 0.3409 - val_accuracy: 0.8783
Epoch 6/34
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2781 - accuracy: 0.8981 - val_loss: 0.3276 - val_accuracy: 0.8802
Epoch 7/34
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2609 - accuracy: 0.9037 - val_loss: 0.3311 - val_accuracy:

To finish, we evaluate the model on the test data.

In [13]:
eval_results = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_results)

313/313 [==============================] - 1s 2ms/step - loss: 0.4703 - accuracy: 0.8841
[test loss, test accuracy]: [0.4703187048435211, 0.8841000199317932]


Now it's time to test the tuner in some other models!